```
You find yourself standing on a snow-covered coastline; apparently, you landed a little off course. The region is too hilly to see the North Pole from here, but you do spot some Elves that seem to be trying to unpack something that washed ashore. It's quite cold out, so you decide to risk creating a paradox by asking them for directions.

"Oh, are you the search party?" Somehow, you can understand whatever Elves from the year 1018 speak; you assume it's Ancient Nordic Elvish. Could the device on your wrist also be a translator? "Those clothes don't look very warm; take this." They hand you a heavy coat.

"We do need to find our way back to the North Pole, but we have higher priorities at the moment. You see, believe it or not, this box contains something that will solve all of Santa's transportation problems - at least, that's what it looks like from the pictures in the instructions." It doesn't seem like they can read whatever language it's in, but you can: "Sleigh kit. Some assembly required."

"'Sleigh'? What a wonderful name! You must help us assemble this 'sleigh' at once!" They start excitedly pulling more parts out of the box.

The instructions specify a series of steps and requirements about which steps must be finished before others can begin (your puzzle input). Each step is designated by a single letter. For example, suppose you have the following instructions:

Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.
Visually, these requirements look like this:


  -->A--->B--
 /    \      \
C      -->D----->E
 \           /
  ---->F-----
Your first goal is to determine the order in which the steps should be completed. If more than one step is ready, choose the step which is first alphabetically. In this example, the steps would be completed as follows:

Only C is available, and so it is done first.
Next, both A and F are available. A is first alphabetically, so it is done next.
Then, even though F was available earlier, steps B and D are now also available, and B is the first alphabetically of the three.
After that, only D and F are available. E is not available because only some of its prerequisites are complete. Therefore, D is completed next.
F is the only choice, so it is done next.
Finally, E is completed.
So, in this example, the correct order is CABDFE.

In what order should the steps in your instructions be completed?
```

In [1]:
import re

PARSE_LINE = re.compile("Step ([A-Z]) must be finished before step ([A-Z]) can begin.")

def parse_line(line):
    return PARSE_LINE.match(line).groups()

class Instructions:    
        
    def __init__(self, ins):          
        self.deps = {}
        self.running = set()
        
        for a, b in ins:
            
            if b not in self.deps:
                self.deps[b] = set()

            if a not in self.deps:
                self.deps[a] = set()

            self.deps[b].add(a)
        
    def ready(self):
        return self.deps |> filter$(x->len(self.deps[x])==0) |> filter$(x->x not in self.running) |> sorted
    
    def start(self, a):
        del self.deps[a]
        self.running.add(a)
        
    def finish(self, a):
        for k in self.deps:
            self.deps[k].discard(a)
        self.running.discard(a)
    
    def __len__(self):
        return len(self.deps)
    
def order(ins):
    
    while len(ins) > 0:
        i = ins.ready()[0]
        yield i
        ins.start(i)
        ins.finish(i)
            

In [2]:
example = [
    "Step C must be finished before step A can begin.",
    "Step C must be finished before step F can begin.",
    "Step A must be finished before step B can begin.",
    "Step A must be finished before step D can begin.",
    "Step B must be finished before step E can begin.",
    "Step D must be finished before step E can begin.",
    "Step F must be finished before step E can begin."
]

In [3]:
Instructions(example |> fmap$(parse_line)) |> order |> "".join

'CABDFE'

In [4]:
Instructions(open("input/day07.txt") |> fmap$(parse_line)) |> order |> "".join

'BGKDMJCNEQRSTUZWHYLPAFIVXO'

```
--- Part Two ---
As you're about to begin construction, four of the Elves offer to help. "The sun will set soon; it'll go faster if we work together." Now, you need to account for multiple people working on steps simultaneously. If multiple steps are available, workers should still begin them in alphabetical order.

Each step takes 60 seconds plus an amount corresponding to its letter: A=1, B=2, C=3, and so on. So, step A takes 60+1=61 seconds, while step Z takes 60+26=86 seconds. No time is required between steps.

To simplify things for the example, however, suppose you only have help from one Elf (a total of two workers) and that each step takes 60 fewer seconds (so that step A takes 1 second and step Z takes 26 seconds). Then, using the same instructions as above, this is how each second would be spent:

Second   Worker 1   Worker 2   Done
   0        C          .        
   1        C          .        
   2        C          .        
   3        A          F       C
   4        B          F       CA
   5        B          F       CA
   6        D          F       CAB
   7        D          F       CAB
   8        D          F       CAB
   9        D          .       CABF
  10        E          .       CABFD
  11        E          .       CABFD
  12        E          .       CABFD
  13        E          .       CABFD
  14        E          .       CABFD
  15        .          .       CABFDE
Each row represents one second of time. The Second column identifies how many seconds have passed as of the beginning of that second. Each worker column shows the step that worker is currently doing (or . if they are idle). The Done column shows completed steps.

Note that the order of the steps has changed; this is because steps now take time to finish and multiple workers can begin multiple steps simultaneously.

In this example, it would take 15 seconds for two workers to complete these steps.

With 5 workers and the 60+ second step durations described above, how long will it take to complete all of the steps?

```

In [5]:
class Workers:
    
    def __init__(self, workers=2, delay=0):
        
        # Total time working
        self.t = 0
        
        # Delay to add to each instruction (Using ASCII code)
        self.delay = delay - ord("A") + 1
        
        # Remaining working time for each worker
        self.wt = [0] * workers
        
        # Current working instructions ids (or '.' if the worker is idle)
        self.wi = ['.'] * workers
        
    # Check if there is at least one worker idle
    def is_busy(self):
        return '.' not in self.wi
    
    # Start an instructions at first idle worker
    def start(self, i):
        idx = self.wt.index(0)
        self.wt[idx] = ord(i) + self.delay
        self.wi[idx] = i    
        
    # Work until all the instructions finish
    # and return the total time
    def finish(self):
        self.t += max(self.wt)
        return self.t        
    
    # Work until at least one instruction finish
    # and return the finished instructions
    def work(self):       
        
        # Workers not idle
        idx = [i for i, t in enumerate(self.wi) if t != '.']        
        
        if len(idx) > 0:
            
            # Minimum time to work
            min_time = min([self.wt[i] for i in idx])
            
            self.t += min_time            
            for i in idx:
                self.wt[i] -= min_time
                
                if self.wt[i] == 0:
                    # Return the finished 
                    fi = self.wi[i]
                    self.wi[i] = '.'
                    yield fi              


def totaltime(ins, workers):
    
    while len(ins) > 0:        
        ready_ins = ins.ready()
        
        # Work if there is nothing ready or all the workers are busy
        if workers.is_busy() or len(ready_ins) == 0:
            for i in workers.work():
                ins.finish(i)
        
        # Start new instructions
        for i in ready_ins:  
            if not workers.is_busy():
                workers.start(i)
                ins.start(i)
    
    return workers.finish()

In [6]:
totaltime(
    Instructions(example |> fmap$(parse_line)),
    Workers()
)

15

In [7]:
totaltime(
    Instructions(open('input/day07.txt') |> fmap$(parse_line)),
    Workers(workers=5, delay=60)
)

941